<h1 align="center">Trabalho 3 - Aprendizagem de Máquina</h1>

## Andre Brun
### Daniel Boll & Mateus Karvat
---

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns; sns.set()

import time

# Analitics
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error
from scipy.stats import pearsonr 

# Linear Models
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression

# Classifiers
from sklearn.svm import SVR

# Regressors
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor

# Configurations
style.use('ggplot')
%matplotlib qt
np.set_printoptions(precision=3, suppress=True)
pd.set_option("display.precision", 3)

In [2]:
data = pd.read_csv('./winequality-red.csv')

# Manteremos uma cópia dos dados originais
# para garantia
raw_data = data.copy()

In [3]:
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.998,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.997,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.997,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.998,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.998,3.51,0.56,9.4,5


In [4]:
data['quality'].value_counts()

5    681
6    638
7    199
4     53
8     18
3     10
Name: quality, dtype: int64

In [5]:
X = data.values[:, :-1]
y = data.values[:, -1]
attributes_size = np.shape(X)[1]

In [6]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y)

In [7]:
each_r = np.zeros((attributes_size, 1))
n = np.shape(X)[0]
k = np.shape(X)[1]
constant = ((n - 1) / (n - k - 1))
for i in range(np.shape(X)[1]):
    rsquared = pearsonr(X[:, i], y)[0] ** 2
    each_r[i] = (1 - (1 - rsquared) * constant)

In [8]:
plt.figure(figsize=(18, 15))
plt.subplots_adjust(hspace=0.40, wspace=0.3)

for i in range(attributes_size):
    coluna = X[:, i]

    current_figure = plt.subplot(3, 4, i+1)
    plt.plot(y, coluna, 'b.') 
    current_figure.set_title(f"{data.columns[i].title()}\n R² ajustado = {np.round(each_r[i][0], 3)}")

plt.suptitle("Correlação de cada atributo com a qualidade", size=30)
plt.show()

In [9]:
attributes = ['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar', 'chlorides', 'free sulfur dioxide','total sulfur dioxide', 'density', 'pH', 'sulphates','alcohol', 'quality']

correlations = data[attributes].corr()

mask = np.zeros_like(data[attributes].corr(), dtype=np.bool) 
mask[np.triu_indices_from(mask)] = True 

f, ax = plt.subplots(figsize=(12, 9))
plt.title("Correlação entre atributos", fontsize=25)

sns.heatmap(correlations, linewidths=.25, vmax=.7, square=True, cmap="YlGn", linecolor="w", annot=True, annot_kws={"size": 13}, mask=mask, cbar_kws={"shrink": .9})

plt.tight_layout()

----
### Classificadores: 
- MultLinReg
- SVM (SRV)
- MLP
- Random Forest

---
### Multiple Linear Regression

In [10]:
mlr = LinearRegression().fit(x_train, y_train)
y_pred = mlr.predict(x_test)

mlr_msq = np.sqrt(mean_squared_error(y_test, y_pred))
mlr_sse = np.sum((y_test - y_pred)**2)

In [11]:
print(" "*10+"MLR")
print("="*23)
print("Root Mean Squared Error")
print("-"*23)
print(f"   {mlr_msq}")
print("="*23)
print("*"*23)
print("="*23)
print("Sum of Squared Errors")
print("-"*23)
print(f"   {mlr_sse}")
print("="*23)

          MLR
Root Mean Squared Error
-----------------------
   0.6657758749910998
***********************
Sum of Squared Errors
-----------------------
   212.763607545679


---
## SVM (Support Vector Machine) 
### SVR

In [12]:
svr = SVR(kernel="linear").fit(x_train, y_train)
y_pred = svr.predict(x_test)
svr_msq = np.sqrt(mean_squared_error(y_test, y_pred))
svr_sse = np.sum((y_test - y_pred)**2)

In [13]:
print(" "*10+"SVR")
print("="*23)
print("Root Mean Squared Error")
print("-"*23)
print(f"   {svr_msq}")
print("="*23)
print("*"*23)
print("="*23)
print("Sum of Squared Errors")
print("-"*23)
print(f"   {svr_sse}")
print("="*23)

          SVR
Root Mean Squared Error
-----------------------
   0.6733032506555411
***********************
Sum of Squared Errors
-----------------------
   217.6018883247928


---
## MLP (Multi Layer Perceptron)


In [14]:
mlp = MLPRegressor(
    hidden_layer_sizes = (256, 168, 64, 32),
    solver="lbfgs",
    activation="tanh",
    max_iter=1500,
    alpha=0,
    ).fit(x_train, y_train)
y_pred = mlp.predict(x_test)

mlp_msq = np.sqrt(mean_squared_error(y_test, y_pred))
mlp_sse = np.sum((y_test - y_pred)**2)

In [15]:
print(" "*10+"MLP")
print("="*23)
print("Root Mean Squared Error")
print("-"*23)
print(f"   {mlp_msq}")
print("="*23)
print("*"*23)
print("="*23)
print("Sum of Squared Errors")
print("-"*23)
print(f"   {mlp_sse}")
print("="*23)

          MLP
Root Mean Squared Error
-----------------------
   0.7798266315140114
***********************
Sum of Squared Errors
-----------------------
   291.9021961048751


---
## Random Forest Regressor

In [16]:
rfr = RandomForestRegressor(n_estimators=300, n_jobs=-1).fit(x_train, y_train)
y_pred = rfr.predict(x_test)

rfr_msq = np.sqrt(mean_squared_error(y_test, y_pred))
rfr_sse = np.sum((y_test - y_pred)**2)

In [17]:
print(" "*10+"RFR")
print("="*23)
print("Root Mean Squared Error")
print("-"*23)
print(f"   {rfr_msq}")
print("="*23)
print("*"*23)
print("="*23)
print("Sum of Squared Errors")
print("-"*23)
print(f"   {rfr_sse}")
print("="*23)

          RFR
Root Mean Squared Error
-----------------------
   0.5917366938215128
***********************
Sum of Squared Errors
-----------------------
   168.0731111111111


---

In [18]:
table = [
    [mlr_msq, svr_msq, mlp_msq, rfr_msq],
    [mlr_sse, svr_sse, mlp_sse, rfr_sse]
]
result = pd.DataFrame(table,
    index=["MSQ", "SSE"],
    columns=["MLR", "SVR", "MLP", "RFR"]
    )
result.style.highlight_min(color = "forestgreen", axis=1)

,MLR,SVR,MLP,RFR
MSQ,0.666,0.673,0.780,0.592
SSE,212.764,217.602,291.902,168.073
